В этом практикуме мы рассмотрим работу с библиотекой **Gensim** для работы с векторными представлениями текста

Мы рассмотрим
- **Word2Vec** - векторные представления слов
- **FastText** - улучшенные представления с учетом морфологии  
- **Doc2Vec** - векторные представления документов


In [ ]:
!pip install gensim

import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import numpy as np

## Часть 1: Word2Vec

### Что такое Word2Vec?

Word2Vec преобразует слова в векторы чисел так, что семантически похожие слова оказываются близко в векторном пространстве.

**Два основных алгоритма:**
- **CBOW** - предсказывает слово по контексту
- **Skip-gram** - предсказывает контекст по слову

**Загрузка предобученной модели**

In [ ]:
w2v_model = api.load('glove-wiki-gigaword-100')

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")

KeyboardInterrupt: 

Найдите документацию `gensim`: какие датасеты кроме `glove-wiki-gigaword-100` доступны в библиотеке?

Выберите 3 датасета и кратко опишите их (источник данных, примерный объем, зачем такой датасет может использоваться)

**Базовые операции с векторами**

In [ ]:
# Получаем вектор слова
vector = w2v_model['computer']
print(f"Вектор слова 'computer': {vector[:5]}...")  # Показываем первые 5 чисел

# Вычисляем схожесть между словами
similarity = w2v_model.similarity('computer', 'laptop')
print(f"Схожесть 'computer' и 'laptop': {similarity:.4f}")

Вектор слова 'computer': [-0.16298   0.30141   0.57978   0.066548  0.45835 ]...
Схожесть 'computer' и 'laptop': 0.7024


**Поиск похожих слов**

In [ ]:
# Находим похожие слова
similar_words = w2v_model.most_similar('python', topn=5)
print("Слова, похожие на 'python':")
for word, score in similar_words:
    print(f"  {word}: {score:.4f}")

Слова, похожие на 'python':
  monty: 0.6886
  php: 0.5865
  perl: 0.5784
  cleese: 0.5447
  flipper: 0.5113


*Ваш ответ здесь*

**Задание**

1. Загрузите любой датасет из gensim на ваш выбор

In [12]:
import gensim.downloader as api

w2v_model = api.load("glove-wiki-gigaword-300")

print(f"Размер словаря: {len(w2v_model.key_to_index)}")
print(f"Размерность векторов: {w2v_model.vector_size}")
print("Пример слов:", list(w2v_model.key_to_index.keys())[:10])

Размер словаря: 400000
Размерность векторов: 300
Пример слов: ['the', ',', '.', 'of', 'to', 'and', 'in', 'a', '"', "'s"]


2. Напишите функцию, которая принимает на вход любое слово и вовращает 10 наиболее близких по вектору слов

In [13]:
def ten_closest(word):
    w = word.strip().lower()
    if w not in w2v_model.key_to_index:
        return "Такого слова нет в словаре модели"
    return w2v_model.most_similar(w, topn=10)

print(ten_closest(input("Введите слово: ")))

Введите слово: girl
[('boy', 0.8272891044616699), ('woman', 0.729641854763031), ('girls', 0.7227292060852051), ('teenager', 0.6509774327278137), ('teenage', 0.6492719054222107), ('mother', 0.6417974829673767), ('boys', 0.6283578872680664), ('child', 0.6229295134544373), ('teen', 0.612524151802063), ('daughter', 0.6050207614898682)]


3. Обучите модель Word2Vec на тестовом датасете из ячейки ниже

Примените следующие настройки:

- размер вектора: 50
- размер окна: 3
- минимальная частота слова: 1
- потоков: 2
- использовать skip-gram

In [14]:
cooking_sentences = [
    ['варить', 'суп', 'овощи', 'морковь', 'картофель'],
    ['жарить', 'курица', 'сковорода', 'масло', 'специи'],
    ['печь', 'хлеб', 'мука', 'дрожжи', 'духовка'],
    ['резать', 'овощи', 'салат', 'помидоры', 'огурцы'],
    ['смешивать', 'ингредиенты', 'тесто', 'яйца', 'молоко'],
    ['варить', 'паста', 'вода', 'соль', 'соус'],
    ['гриль', 'мясо', 'овощи', 'уголь', 'барбекю'],
    ['тушить', 'говядина', 'горшок', 'вино', 'травы'],
    ['запекать', 'рыба', 'лимон', 'духовка', 'фольга'],
    ['готовить', 'завтрак', 'яичница', 'бекон', 'тост'],
    ['месить', 'тесто', 'пирог', 'начинка', 'яблоки'],
    ['кипятить', 'вода', 'чай', 'кофе', 'чашка'],
    ['мариновать', 'мясо', 'соус', 'специи', 'холодильник'],
    ['взбивать', 'сливки', 'сахар', 'десерт', 'торт'],
    ['парить', 'овощи', 'здоровое', 'питание', 'брокколи']
]

In [20]:

model = Word2Vec(
    sentences=cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    sg=1
)

print("Готово! Модель обучена.")
print("Первые 10 слов в словаре:", list(model.wv.key_to_index.keys())[:10])

Готово! Модель обучена.
Первые 10 слов в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']


In [21]:
print(f"Слова в словаре: {list(model.wv.key_to_index.keys())[:10]}...")

Слова в словаре: ['овощи', 'мясо', 'соус', 'вода', 'тесто', 'духовка', 'специи', 'варить', 'брокколи', 'питание']...


4. Проверьте модель

In [22]:
# Проверяем похожие слова в кулинарной тематике
try:
    similar = model.wv.most_similar('варить', topn=5)
    print("Слова, похожие на 'варить':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'варить' не найдено в словаре")

Слова, похожие на 'варить':
  вино: 0.2398
  ингредиенты: 0.2172
  хлеб: 0.1938
  брокколи: 0.1846
  кипятить: 0.1711


In [23]:
# Найдите слова, похожие на "духовка"
try:
    similar = model.wv.most_similar('духовка', topn=10)
    print("Слова, похожие на 'духовка':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'духовка' не найдено в словаре")

print()

# Найдите слова, похожие на "овощи"
try:
    similar = model.wv.most_similar('овощи', topn=10)
    print("Слова, похожие на 'овощи':")
    for word, score in similar:
        print(f"  {word}: {score:.4f}")
except KeyError:
    print("Слово 'овощи' не найдено в словаре")

Слова, похожие на 'духовка':
  ингредиенты: 0.3199
  десерт: 0.3064
  холодильник: 0.2705
  питание: 0.2243
  пирог: 0.2142
  яблоки: 0.2049
  парить: 0.1978
  специи: 0.1834
  вино: 0.1411
  жарить: 0.1394

Слова, похожие на 'овощи':
  мариновать: 0.2716
  хлеб: 0.2691
  гриль: 0.2546
  фольга: 0.2409
  сахар: 0.2108
  жарить: 0.1968
  яблоки: 0.1862
  говядина: 0.1759
  парить: 0.1670
  кипятить: 0.1610


## Часть 2: FastText

FastText улучшает Word2Vec, рассматривая слова как наборы символов (n-грамм). Это позволяет работать с редкими словами и опечатками

5. Обучите FastText на корпусе текстов из пункта 3. Используйте код ниже

In [25]:
ft_model = FastText(
    sentences= cooking_sentences,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2
)



6. Найдите слова, похожие на "варить", "духовка" и "овощи" с помощью обученной модели. Используйте код из пункта 4

In [26]:
def print_similar_ft(word, topn=10):
    try:
        sims = ft_model.wv.most_similar(word, topn=topn)
        print(f"Слова, похожие на '{word}':")
        for w, score in sims:
            print(f"  {w}: {score:.4f}")
        print()
    except KeyError:
        print(f"Слово '{word}' не найдено в словаре FastText\n")

print_similar_ft("варить", topn=10)
print_similar_ft("духовка", topn=10)
print_similar_ft("овощи", topn=10)

Слова, похожие на 'варить':
  жарить: 0.5353
  парить: 0.4805
  месить: 0.3541
  тушить: 0.3405
  специи: 0.2622
  рыба: 0.2425
  бекон: 0.2348
  запекать: 0.2155
  завтрак: 0.2032
  смешивать: 0.1994

Слова, похожие на 'духовка':
  взбивать: 0.4565
  лимон: 0.3561
  салат: 0.3050
  курица: 0.3041
  тост: 0.2944
  кофе: 0.2896
  месить: 0.2676
  говядина: 0.2456
  масло: 0.2428
  рыба: 0.2423

Слова, похожие на 'овощи':
  жарить: 0.2960
  фольга: 0.2574
  морковь: 0.2297
  соус: 0.2172
  торт: 0.2094
  здоровое: 0.1955
  ингредиенты: 0.1942
  кипятить: 0.1651
  сахар: 0.1448
  барбекю: 0.1434



7. Сравните модели

Дана функция для сравнения Word2Vec и FastText

Придумайте 3 слова с опечатками и проверьте, найдет ли их FastText и Word2Vec

In [27]:
def compare_models(word):
    """Сравнивает представления слова в разных моделях"""
    print(f"\nСравнение для слова: '{word}'")

    # Word2Vec
    try:
        w2v_similar = model.wv.most_similar(word, topn=2)
        print(f"  Word2Vec: {[w for w, _ in w2v_similar]}")
    except KeyError:
        print(f"  Word2Vec: слово не найдено")

    # FastText
    try:
        ft_similar = ft_model.wv.most_similar(word, topn=2)
        print(f"  FastText: {[w for w, _ in ft_similar]}")
    except KeyError:
        print(f"  FastText: слово не найдено")

# Сравниваем для разных слов
compare_models('борбекю')
compare_models('сус')


Сравнение для слова: 'борбекю'
  Word2Vec: слово не найдено
  FastText: ['барбекю', 'говядина']

Сравнение для слова: 'сус'
  Word2Vec: слово не найдено
  FastText: ['лимон', 'фольга']


## Часть 3: Doc2Vec

Doc2Vec расширяет Word2Vec для создания векторных представлений целых документов (предложений, абзацев, статей)

In [28]:
# Создаем размеченные документы
documents = [
    "machine learning is interesting",
    "deep learning uses neural networks",
    "python programming for data science",
    "artificial intelligence is amazing",
    "computer vision processes images"
]

# Преобразуем в формат TaggedDocument
tagged_docs = []
for i, doc in enumerate(documents):
    tokens = doc.split()
    tagged_doc = TaggedDocument(words=tokens, tags=[f"doc_{i}"])
    tagged_docs.append(tagged_doc)

print("Размеченные документы:")
for doc in tagged_docs[:3]:
    print(f"  Слова: {doc.words}")
    print(f"  Тег: {doc.tags}")

Размеченные документы:
  Слова: ['machine', 'learning', 'is', 'interesting']
  Тег: ['doc_0']
  Слова: ['deep', 'learning', 'uses', 'neural', 'networks']
  Тег: ['doc_1']
  Слова: ['python', 'programming', 'for', 'data', 'science']
  Тег: ['doc_2']


In [29]:
# Обучаем Doc2Vec
doc_model = Doc2Vec(
    documents=tagged_docs,
    vector_size=50,
    window=3,
    min_count=1,
    workers=2,
    epochs=20
)

print("Doc2Vec модель обучена!")
print(f"Количество документов: {len(doc_model.dv.key_to_index)}")

Doc2Vec модель обучена!
Количество документов: 5


In [30]:
# Получаем вектор документа
doc_vector = doc_model.dv["doc_0"]
print(f"Вектор документа doc_0: {doc_vector[:5]}...")

# Находим похожие документы
similar_docs = doc_model.dv.most_similar("doc_0", topn=2)
print("\nДокументы, похожие на doc_0:")
for doc_tag, similarity in similar_docs:
    doc_id = int(doc_tag.split('_')[1])
    print(f"  {doc_tag}: {similarity:.4f}")
    print(f"    Текст: {documents[doc_id]}")

Вектор документа doc_0: [-0.01057    -0.01198188 -0.01982618  0.01710627  0.00710373]...

Документы, похожие на doc_0:
  doc_1: 0.2735
    Текст: deep learning uses neural networks
  doc_2: 0.1275
    Текст: python programming for data science


In [31]:
# Сравниваем схожесть документов
def compare_documents(doc1_id, doc2_id):
    similarity = doc_model.dv.similarity(f"doc_{doc1_id}", f"doc_{doc2_id}")
    print(f"Схожесть doc_{doc1_id} и doc_{doc2_id}: {similarity:.4f}")
    print(f"  doc_{doc1_id}: {documents[doc1_id]}")
    print(f"  doc_{doc2_id}: {documents[doc2_id]}")

compare_documents(0, 1)  # machine learning vs deep learning
compare_documents(0, 3)  # machine learning vs AI

Схожесть doc_0 и doc_1: 0.2735
  doc_0: machine learning is interesting
  doc_1: deep learning uses neural networks
Схожесть doc_0 и doc_3: -0.0822
  doc_0: machine learning is interesting
  doc_3: artificial intelligence is amazing


8. Сравните схожесть doc_2 и doc_4

In [32]:
compare_documents(2, 4)

Схожесть doc_2 и doc_4: -0.0362
  doc_2: python programming for data science
  doc_4: computer vision processes images


9. Найдите самый похожий документ на doc_1

In [36]:
doc_ids = range(len(documents))
max_similarity = -1
most_similar_doc = None

for i in doc_ids:
    if i == 1:
        continue
    sim = compare_documents(1, i)
    if sim > max_similarity:
        max_similarity = sim
        most_similar_doc = i

print(f"\nНаиболее похожий документ на doc_1 — doc_{most_similar_doc}, similarity: {max_similarity:.4f}")
print("Текст doc_1:", documents[1])
print(f"Текст doc_{most_similar_doc}:", documents[most_similar_doc])

Схожесть doc_1 и doc_0: 0.2735
  doc_1: 1
  doc_0: 0
Схожесть doc_1 и doc_2: -0.0573
  doc_1: 1
  doc_2: 2
Схожесть doc_1 и doc_3: 0.2031
  doc_1: 1
  doc_3: 3
Схожесть doc_1 и doc_4: -0.2546
  doc_1: 1
  doc_4: 4

Наиболее похожий документ на doc_1 — doc_0, similarity: 0.2735
Текст doc_1: 1
Текст doc_0: 0


10. Выберите любую из трёх моделей. Обучите модели с разной размерностью (10, 50, 100). Продемонстрируйте качество их работы на примере поиска похожих слов (выберите любые 3 примера, соответствующих тематике корпуса из пункта 4)

In [37]:
from gensim.models import Word2Vec

sizes = [10, 50, 100]
test_words = ["варить", "духовка", "овощи"]

models_by_size = {}

for sz in sizes:
    m = Word2Vec(
        sentences=cooking_sentences,
        vector_size=sz,
        window=3,
        min_count=1,
        workers=2,
        sg=1
    )
    models_by_size[sz] = m
    print(f" Обучили Word2Vec: vector_size={sz}")

print("\n" + "="*60)

for word in test_words:
    print(f"\nСлово: '{word}'")
    for sz in sizes:
        m = models_by_size[sz]
        try:
            sims = m.wv.most_similar(word, topn=5)
            sims_words = [w for w, _ in sims]
            print(f"  size={sz}: {sims_words}")
        except KeyError:
            print(f"  size={sz}: слова нет в словаре")

 Обучили Word2Vec: vector_size=10
 Обучили Word2Vec: vector_size=50
 Обучили Word2Vec: vector_size=100


Слово: 'варить'
  size=10: ['рыба', 'сковорода', 'хлеб', 'готовить', 'яичница']
  size=50: ['вино', 'ингредиенты', 'хлеб', 'брокколи', 'кипятить']
  size=100: ['чашка', 'вино', 'сковорода', 'суп', 'мука']

Слово: 'духовка'
  size=10: ['взбивать', 'тушить', 'говядина', 'кипятить', 'лимон']
  size=50: ['ингредиенты', 'десерт', 'холодильник', 'питание', 'пирог']
  size=100: ['жарить', 'парить', 'завтрак', 'тесто', 'вода']

Слово: 'овощи'
  size=10: ['жарить', 'фольга', 'горшок', 'уголь', 'ингредиенты']
  size=50: ['мариновать', 'хлеб', 'гриль', 'фольга', 'сахар']
  size=100: ['взбивать', 'питание', 'бекон', 'резать', 'гриль']
